In [1]:
# Loading the Reuters dataset
from keras.datasets import reuters

(train_data, train_labels), (test_data, test_labels) = reuters.load_data(num_words=10000)

/dados/anaconda3/lib/python3.8/site-packages/keras/datasets/reuters.py:143: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])
/dados/anaconda3/lib/python3.8/site-packages/keras/datasets/reuters.py:144: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_test, y_test = np.array(xs[idx:]), np.array(labels[idx:])


In [2]:
# Encoding the data
import numpy as np

def vectorize_sequences(sequences, dimension=10000):
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1.
    return results

x_train = vectorize_sequences(train_data)
x_test = vectorize_sequences(test_data)

In [3]:
# there is a built-in way to do this in Keras
from keras.utils.np_utils import to_categorical

one_hot_train_labels = to_categorical(train_labels)
one_hot_test_labels = to_categorical(test_labels)

In [4]:
import os
os.environ['PYTHONHASHSEED']=str(1)

import tensorflow as tf
import random
import numpy as np

def reset_random_seeds(SEED = 42):
  os.environ['PYTHONHASHSEED']=str(SEED)
  tf.random.set_seed(SEED)
  np.random.seed(SEED)
  random.seed(SEED)

In [5]:
from keras import models
from keras import layers

def create_and_train_model(hidden_layers=2,
                           hidden_units=64,
                           loss_function='categorical_crossentropy',
                           activation_function='relu'):
  
  model = models.Sequential()

  model.add(layers.Dense(hidden_units, activation=activation_function, input_shape=(10000,)))

  for i in range(hidden_layers - 1):
    model.add(layers.Dense(hidden_units, activation=activation_function))
 
  model.add(layers.Dense(46, activation='softmax'))

  model.compile(optimizer='rmsprop',
                loss=loss_function,
                metrics=['accuracy'])

  model.fit(x_train,
            one_hot_train_labels,
            epochs=8,
            verbose=0,
            batch_size=512)

  results = model.evaluate(x_test, one_hot_test_labels)

  return "[hidden_layers=%d, hidden_units=%d, loss_function=%s, " \
          "activation_function=%s]\n => loss: %.4f - accuracy: %.4f\n" % (
          hidden_layers, hidden_units, loss_function, activation_function,
          results[0], results[1])

In [6]:
%%time

# Try using larger or smaller layers: 32 units, 128 units, and so on.

# => best result: 512 hidden units

for value in [16, 32, 64, 128, 256, 512]:
  res = create_and_train_model(hidden_units=value)
  print(res)

71/71 [==============================] - 0s 604us/step - loss: 1.2913 - accuracy: 0.7026
[hidden_layers=2, hidden_units=16, loss_function=categorical_crossentropy, activation_function=relu]
 => loss: 1.2913 - accuracy: 0.7026

71/71 [==============================] - 0s 660us/step - loss: 1.0421 - accuracy: 0.7663
[hidden_layers=2, hidden_units=32, loss_function=categorical_crossentropy, activation_function=relu]
 => loss: 1.0421 - accuracy: 0.7663

71/71 [==============================] - 0s 796us/step - loss: 0.9641 - accuracy: 0.7885
[hidden_layers=2, hidden_units=64, loss_function=categorical_crossentropy, activation_function=relu]
 => loss: 0.9641 - accuracy: 0.7885

71/71 [==============================] - 0s 1ms/step - loss: 0.9903 - accuracy: 0.7872
[hidden_layers=2, hidden_units=128, loss_function=categorical_crossentropy, activation_function=relu]
 => loss: 0.9903 - accuracy: 0.7872

71/71 [==============================] - 0s 2ms/step - loss: 1.0730 - accuracy: 0.7903
[hidde

In [8]:
%%time

# You used two hidden layers. Now try using a single hidden layer, or three hidden layers.

# => best result: 1 hidden layer(s)

for value in [1, 2, 3, 4]:
  res = create_and_train_model(hidden_layers=value)
  print(res)

71/71 [==============================] - 0s 764us/step - loss: 0.8708 - accuracy: 0.8054
[hidden_layers=1, hidden_units=64, loss_function=categorical_crossentropy, activation_function=relu]
 => loss: 0.8708 - accuracy: 0.8054

71/71 [==============================] - 0s 815us/step - loss: 1.0033 - accuracy: 0.7858
[hidden_layers=2, hidden_units=64, loss_function=categorical_crossentropy, activation_function=relu]
 => loss: 1.0033 - accuracy: 0.7858

71/71 [==============================] - 0s 783us/step - loss: 1.0395 - accuracy: 0.7827
[hidden_layers=3, hidden_units=64, loss_function=categorical_crossentropy, activation_function=relu]
 => loss: 1.0395 - accuracy: 0.7827

71/71 [==============================] - 0s 827us/step - loss: 1.2127 - accuracy: 0.7507
[hidden_layers=4, hidden_units=64, loss_function=categorical_crossentropy, activation_function=relu]
 => loss: 1.2127 - accuracy: 0.7507

CPU times: user 46.5 s, sys: 1.82 s, total: 48.3 s
Wall time: 8.41 s
